<a href="https://colab.research.google.com/github/bxck75/piss-ant-pix2pix/blob/master/modeltransferv3_googlescrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# remove defaults
!rm -r sample_data
# Clone the repo
!git clone https://github.com/bxck75/Python_Helpers.git

# Change dir
%cd /content/Python_Helpers
# install
!python setup.py install

from IPython.display import clear_output
import main
import os
import sys
import IPython
import Helpers
# get pix2pix repo
P=Helpers.core.Core()
hlp=P.H
%cd /content/
hlp.repolist= hlp.repo_collection
repos_sorted = hlp.repo_collection.repos_sorted
A1=['bxck75/piss-ant-pix2pix']
hlp.Me(['inst_reps',A1,'/content/installed_repos',False,True])
clear_output()

In [0]:
!rm -r /content/images

In [0]:
# %cd /content/
# dir (Helpers.core.GoImgScrape.GoogleImageCrawler)
# img_crawler = Helpers.core.GoImgScrape.GoogleImageCrawler()
# help(img_crawler)
# search='orc face'
# img_crawler.crawl(search,max_num=200)

In [0]:
%cd /content/
'''Crawl'''
dir (Helpers.core.GoImgScrape.GoogleImageCrawler)
img_crawler = Helpers.core.GoImgScrape.GoogleImageCrawler()
help(img_crawler)
search='orc face'
img_crawler.crawl(search,max_num=50)
''' prep images '''
images_set_name= search.replace(' ','_')
!mkdir -p {images_set_name}
!cp /content/images/* /content/{images_set_name}/
# !rm -r /content/images
''' zip new pack to gdrive '''
%cd /content/
Helpers.Core()
obj=Helpers.core.Core()
folder_of_model='/content/'+images_set_name
result=obj.H.zip(images_set_name,obj.Gdrive_root,folder_of_model).ZipUp
zip_hash=result.split('(id) ')[1]
'''remove the image folder'''
# !rm -r /content/{images_set_name}
images_set_code=zip_hash
print(images_set_name)
print(images_set_code)
'''Download the image folder'''
Helpers.GdriveD.GdriveD(images_set_code, '/content/'+images_set_name+'_in.zip')
!unzip /content/{images_set_name}_in.zip -d {images_set_name}_in
'''make the set'''
%cd /content/installed_repos/piss-ant-pix2pix
!bash image_folder_make_set_train_colab.sh /content/{images_set_name} 1 prep
# clear_output()

In [0]:
# remove
!rm  -r /content/p2p_process_tmp
!rm -r /content/orc_faceimages
!rm -r /content/orc_face_resize
!rm /content/orc_face.zip

In [0]:
# get images
%cd /content/
# images_set_name='insect'
# images_set_code='1fbEUcBIvLwjlVFrUiW2g8KtRTvFSsW9D'
# images_model_code='1eFR5ZHfp5PzPEnFv_v3xxE9T0tKPJTS4'

# images_set_name='apes'
# images_set_code='1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv'
# images_model_code='1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9'

# images_set_name='garbagepailkids'
# images_set_code='1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-'
# images_model_code='1tp8Gm1fMuSQoLEEXQ_C1Dlw1N4wwFxKb'
%cd /content/
Helpers.GdriveD.GdriveD(images_set_code, '/content/'+images_set_name+'.zip')
!unzip /content/{images_set_name}.zip 
%cd /content/installed_repos/piss-ant-pix2pix
!bash image_folder_make_set_train_colab.sh /content/{images_set_name} 1 prep
clear_output()

In [0]:
import sys
import os
import dlib
import glob
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
''' 
    loop  1x
    1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
for i in range(1):
    ''' fetch metrics'''
    %cd /content/
    # Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    # !unzip /content/{images_set_name}_model_in.zip -d metrics/

    ''' train epochs '''
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --output_dir /content/metrics --progress_freq 110 --save_freq 150 --summary_freq 0 --display_freq 10 --max_epochs 1 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'

    %cd /content/
    '''clean old models,test images,logs etc befor zipping'''
    # delete unwanted
    !rm -r /content/metrics/index.html

    # clean up images
    img_list=hlp.Me(['globx','/content/metrics/images','*.*g'])
    img_list = sorted(img_list)
    print(img_list)
    n=6 #pop the last 6 items off the list (latest images)
    latest = img_list[-n:]
    for i in range(n):
        img = dlib.load_rgb_image(latest[i]) 
        plt.imshow(img)
        plt.show()
    del img_list[-n:]
    # delete the files left in the list
    for i_file in img_list:
        img = dlib.load_rgb_image(i_file) 
        plt.imshow(img)
        plt.show()
        print('deleting : ' + i_file)
        !echo rm -r {i_file}

    # clean up models
    models_list=hlp.Me(['globx','/content/metrics','model-*'])
    models_list = sorted(models_list)
    print(models_list)
    n=6 #pop the last 3 items off the list (latest model)
    del models_list[-n:]
    # delete the files left in the list
    for m_file in models_list:
        print('deleting : ' + m_file)
        !echo rm -r {m_file}

    # clean up events
    events_list=hlp.Me(['globx','/content/metrics','events*'])
    events_list = sorted(events_list)
    print(events_list)
    n=3 #pop the last item off the list (latest event) 
    del events_list[-n:]
    # delete the files left in the list
    for e_file in events_list:
        print('deleting : ' + e_file)
        !echo rm -r {e_file}

    ''' zip metrics up to gdrive '''
    %cd /content/
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)
    
    # delete old metrics before unpacking new in the beginning of the loop
    !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

In [0]:
images_model_code = zip_hash
print(images_model_code)
# !rm -r /content/metrics

In [0]:
def loadTboard():
    '''load tensorboard'''
    import datetime, os ,tensorboard
    # install tensorboard
    #     hlp.Me(['cml','pip install -q tensorflow'])
    # Load the TensorBoard notebook extension
    try:
        # %load_ext tensorboard
        %load_ext tensorboard.notebook
    except:
        %reload_ext tensorboard.notebook
loadTboard()

In [0]:
# %tensorboard --logdir /content/metrics

start values
progress  epoch 1  step 10  image/sec 0.7  remaining 122m
dloss_GAN 0.11989411
gloss_GAN 0.060013566
gloss_L1 0.043864187

In [0]:
import sys
import os
import dlib
import glob
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
''' 
    loop  10x
    1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
%tensorboard --logdir /content/metrics
for i in range(1):
    ''' fetch metrics'''
    %cd /content/
    Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    !unzip /content/{images_set_name}_model_in.zip -d metrics/

    ''' train epochs '''
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --checkpoint /content/metrics --output_dir /content/metrics --progress_freq 50 --save_freq 75 --summary_freq 10 --display_freq 10 --max_epochs 3 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'

    %cd /content/
    '''clean old models,test images,logs etc befor zipping'''
    # delete unwanted
    !rm -r /content/metrics/index.html

    # clean up images
    img_list=hlp.Me(['globx','/content/metrics/images','*.*g'])
    img_list = sorted(img_list)
    print(img_list)
    n=6 #pop the last 6 items off the list (latest images)
    latest = img_list[-n:]
    for i in range(n):
        img = dlib.load_rgb_image(latest[i]) 
        plt.imshow(img)
        plt.show()
    del img_list[-n:]
    # delete the files left in the list
    for i_file in img_list:
        img = dlib.load_rgb_image(i_file) 
        plt.imshow(img)
        plt.show()
        print('deleting : ' + i_file)
        !rm -r {i_file}

    # clean up models
    models_list=hlp.Me(['globx','/content/metrics','model-*'])
    models_list = sorted(models_list)
    print(models_list)
    n=6 #pop the last 6 items off the list (latest model)
    del models_list[-n:]
    # delete the files left in the list
    for m_file in models_list:
        print('deleting : ' + m_file)
        !echo rm -r {m_file}
    # clean up events
    events_list=hlp.Me(['globx','/content/metrics','events*'])
    events_list = sorted(events_list)
    print(events_list)
    n=3 #pop the last item off the list (latest event) 
    del events_list[-n:]
    # delete the files left in the list
    for e_file in events_list:
        print('deleting : ' + e_file)
        !echo rm -r {e_file}

    ''' zip metrics up to gdrive '''
    %cd /content/
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)

    # delete old metrics before unpacking new in the beginning of the loop
    # !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

In [0]:
# !rm -r /content/metrics

In [0]:
# # rewrite checkpointfile
# !echo 'model_checkpoint_path: "/content/metrics/model-21880"' > /content/metrics/checkpoint
# !echo 'all_model_checkpoint_paths: "/content/metrics/model-21880"' >> /content/metrics/checkpoint

In [0]:
# download/unzip metrics 
# Get pretrained metrics
# %cd /content/
# Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
# !unzip /content/{images_set_name}_model_in.zip -d metrics/

In [0]:
# # remove old crap before rezipping
# !rm -r /content/metrics/images
# !rm -r /content/metrics/events.out.tfevents*
# !rm -r /content/metrics/index.html
hlp.Me(['globx','/content/metrics','model-*'])

In [0]:
# # rezip
# %cd /content/
# Helpers.Core()
# obj=Helpers.core.Core()
# folder_of_model='metrics'
# print(obj.Gdrive_root+'/models')
# result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# # print(result)
# zip_hash=result.split('(id) ')[1]
# print(zip_hash)

In [0]:
# test the current checkpoint
# get metrics
%cd /content/
Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
!unzip /content/{images_set_name}_model_in.zip -d metrics/
# test 
%cd /content/installed_repos/piss-ant-pix2pix
!python pix2pix.py --mode test --output_dir /content/{images_set_name}_pretrained/test --input_dir /content/{images_set_name}images/_combined/val --max_steps 100 --which_direction "BtoA" --seed 0 --checkpoint /content/metrics
# export
!python pix2pix.py --mode export --output_dir /content/{images_set_name}_pretrained/exp --checkpoint /content/metrics


In [0]:

# rezip clean export backup
%cd /content/
Helpers.Core()
obj=Helpers.core.Core()
folder_of_model=images_set_name+'_pretrained/exp'
print(obj.Gdrive_root+'/models')
result=obj.H.zip(images_set_name+'model_exp',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# print(result)
zip_hash=result.split('(id) ')[1]
print(zip_hash)

In [0]:
# test exported path
%cd /content/installed_repos/piss-ant-pix2pix
!python pix2pix.py --mode test --output_dir /content/{images_set_name}_pretrained/exp_test --input_dir /content/{images_set_name}images/_combined/val --checkpoint /content/garbagepailkids_pretrained_test --max_steps 100 --which_direction "BtoA" --seed 0 --checkpoint /content/metrics
